In [1]:
from astropy.io import fits
import glob as glob
import numpy as np
import os
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib qt

In [2]:
# read in files, dark-subtract, populate wavelength array

In [3]:
stem = '/Users/bandari/Documents/git.repos/photonics_spectroscopy/notebooks_for_development/data/19pl/raw/'

In [4]:
rel_pos = {'0':(39,81),
           '1':(29,115),
           '2':(38,113),
           '3':(46,111),
           '4':(23,108),
           '5':(32,106),
           '6':(40,105),
           '7':(49,103),
           '8':(17,101),
           '9':(26,100),
           '10':(34,98),
           '11':(43,96),
           '12':(51,94),
           '13':(20,93),
           '14':(28,91),
           '15':(37,90),
           '16':(45,88),
           '17':(22,84),
           '18':(31,83)}

In [5]:
file_list = glob.glob(stem + '19PL_1*fits')

In [6]:
# read in dark
dark_apapane = stem + 'apapanedark.fits'
dark = fits.open(dark_apapane)[0].data

In [7]:
# define reference frame (needs to have all ports visible on array)
file_ref = fits.open(stem + '19PL_1180.fits')[0].data
# dark-subt
ref_frame = file_ref-dark

In [31]:
plt.imshow(array_ref)
plt.show()

In [10]:
df = pd.DataFrame()

In [11]:
dict_info = {'x_shift':[2],'y_shift':[3],'wavel':[2]}
df_this = pd.DataFrame.from_dict(dict_info)

In [12]:
df_this

,x_shift,y_shift,wavel
0,2,3,2


In [13]:
df = pd.concat([df,df_this], ignore_index=True)

In [14]:
df

,x_shift,y_shift,wavel
0,2,3,2


In [15]:
height, width = dark.shape[:2]

for file_num in range(0,len(file_list)):
    
    wavel_this = os.path.basename(file_list[file_num]).split('.fits')[0].split('_')[1]
    
    
    array_this = fits.open(file_list[file_num])[0].data
    
    array_dark_subted = np.subtract(array_this,dark)
    
    # cross-correlate to see shift with respect to reference frame
    corr = scipy.signal.correlate2d(ref_frame, array_this, boundary='symm', mode='same')
    y_abs, x_abs = np.unravel_index(np.argmax(corr), corr.shape)  # y, x: tonight's image is displaced by this much from the reference
    y_shift, x_shift = 0.5*height-y_abs,0.5*width-x_abs
    
    dict_info = {'x_shift':[x_shift],'y_shift':[y_shift],'wavel':[wavel_this]}
    df_this = pd.DataFrame.from_dict(dict_info)
    
    df = pd.concat([df,df_this], ignore_index=True)
    
    plt.scatter([x_shift],[y_shift])
    print(wavel_this)

1375
1725
1230


KeyboardInterrupt: 

In [58]:
df2 = df

In [60]:
df1 = df2.drop([0,1])

In [64]:
df1.to_pickle('junk.pkl')

In [26]:
plt.imshow(array_dark_subted, origin='lower')
for key, xycoord  in rel_pos.items():
    plt.scatter(xycoord[0],xycoord[1])
    plt.annotate(key, xy=(xycoord))
plt.show()

2023-06-12 13:09:17.504 python[17174:190486] +[CATransaction synchronize] called within transaction


In [65]:
plt.imshow(array_dark_subted, origin='lower')
plt.scatter(df1['x_shift'],df1['y_shift'])
plt.show()

In [ ]:
rel_pos = {'0':(39,81),
           '1':(29,115),
           '2':(38,113),
           '3':(46,111),
           '4':(23,108),
           '5':(32,106),
           '6':(40,105),
           '7':(49,103),
           '8':(17,101),
           '9':(26,100),
           '10':(34,98),
           '11':(43,96),
           '12':(51,94),
           '13':(20,93),
           '14':(28,91),
           '15':(37,90),
           '16':(45,88),
           '17':(22,84),
           '18':(31,83)}

In [ ]:
# a test wavelength soln

In [ ]:
def func(X, a, b, c, d, f):
    
    x_pass, y_pass = X
    
    return a*x_pass + b*y_pass + c*x_pass*y_pass + d*np.power(x_pass,2.) + f*np.power(y_pass,2.)

x = df1['x'].values
y = df1['y'].values
z = df1['wavel'].values

# initial guesses for a,b,c:
p0 = 1., 1., 1., 1., 1.
fit_coeffs = curve_fit(func, (x,y), z, p0)

In [ ]:
# column of best-fit wavelength values

df1['wavel_bestfit'] = func((x,y), fit_coeffs[0][0], fit_coeffs[0][1], fit_coeffs[0][2], 
                                     fit_coeffs[0][3], fit_coeffs[0][4])